In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,4)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs
os.environ["RAPIDS_NO_INITIALIZE"] = "1" #Vjawa: Added

In [3]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb
import gc
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,
                               device_memory_limit='10GB', 
                               jit_unspill=True)
                              # protocol="ucx", 
                              # rmm_pool_size="31GB")
    client = Client(cluster)
    return client


In [4]:
client = get_cluster()
client

10.120.104.12


2023-04-13 18:31:26,258 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:31:26,259 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 4
Total threads: 4,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: tcp://10.120.104.12:43189,Workers: 4
Dashboard: http://10.120.104.12:8787/status,Total threads: 4
Started: Just now,Total memory: 1.48 TiB
Comm: tcp://10.120.104.12:35543,Total threads: 1
Dashboard: http://10.120.104.12:44073/status,Memory: 377.64 GiB
Nanny: tcp://10.120.104.12:45975,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq')#.persist()#,split_row_groups=True)
#users.head()

CPU times: user 944 ms, sys: 927 ms, total: 1.87 s
Wall time: 1.93 s


In [6]:
VER = 186
USE = 'clicks'

In [7]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [8]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])
del users

CPU times: user 3.27 s, sys: 1.2 s, total: 4.47 s
Wall time: 58.7 s


In [9]:
gc.collect()

136

In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

/raid/data/mambaforge/envs/rapids-23.04/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-39abb502-da64-11ed-b8fe-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:39726
INFO:distributed.scheduler:Receive client connection: Client-worker-39ac588f-da64-11ed-b904-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:39728
INFO:distributed.scheduler:Receive client connection: Client-worker-39bb0dc6-da64-11ed-b906-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:39734
INFO:distributed.scheduler:Receive client connection: Client-worker-39bf617f-da64-11ed-b901-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:39738
[18:32:31] task [xgboost.dask-0]:tcp://10.120.104.

[0]	train-map:0.16599
[1]	train-map:0.17583
[2]	train-map:0.17589
[3]	train-map:0.17448
[4]	train-map:0.17791
[5]	train-map:0.17842
[6]	train-map:0.17870
[7]	train-map:0.17887
[8]	train-map:0.17817
[9]	train-map:0.17821
[10]	train-map:0.18336
[11]	train-map:0.18549
[12]	train-map:0.18531
[13]	train-map:0.18744
[14]	train-map:0.18773
[15]	train-map:0.18813
[16]	train-map:0.18826
[17]	train-map:0.18843
[18]	train-map:0.18887
[19]	train-map:0.18924
[20]	train-map:0.18931
[21]	train-map:0.18982
[22]	train-map:0.19015
[23]	train-map:0.19029
[24]	train-map:0.19033
[25]	train-map:0.19047
[26]	train-map:0.19045
[27]	train-map:0.19065
[28]	train-map:0.19078
[29]	train-map:0.19108
[30]	train-map:0.19145
[31]	train-map:0.19182
[32]	train-map:0.19235
[33]	train-map:0.19235
[34]	train-map:0.19313
[35]	train-map:0.19325
[36]	train-map:0.19351
[37]	train-map:0.19448
[38]	train-map:0.19522
[39]	train-map:0.19522
[40]	train-map:0.19562
[41]	train-map:0.19614
[42]	train-map:0.19630
[43]	train-map:0.1963

In [12]:
duration = time()-start
duration

270.45068311691284